In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('NY property data.csv')
data.head()

,RECORD,BBLE,BORO,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


In [3]:
remove_list=data['OWNER'].value_counts().head(20).index.tolist()
remove_list

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'YORKVILLE TOWERS ASSO',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 '434 M LLC',
 'LINCOLN PLAZA ASSOCIA',
 'DEUTSCHE BANK NATIONA',
 '561 11TH AVENUE TMG L']

In [4]:
remove_list.remove('YORKVILLE TOWERS ASSO')
remove_list.remove('434 M LLC')
remove_list.remove('DEUTSCHE BANK NATIONA')
remove_list.remove('561 11TH AVENUE TMG L')
remove_list.append('UNITED STATES OF AMER')
remove_list.append('U S GOVERNMENT OWNRD')
remove_list.append('THE CITY OF NEW YORK')
remove_list.append('NYS URBAN DEVELOPMENT')
remove_list.append('NYS DEPT OF ENVIRONME')
remove_list.append('CULTURAL AFFAIRS')
remove_list.append('DEPT OF GENERAL SERVI')
remove_list.append('DEPT RE-CITY OF NY')

In [5]:
remove_list

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 'LINCOLN PLAZA ASSOCIA',
 'UNITED STATES OF AMER',
 'U S GOVERNMENT OWNRD',
 'THE CITY OF NEW YORK',
 'NYS URBAN DEVELOPMENT',
 'NYS DEPT OF ENVIRONME',
 'CULTURAL AFFAIRS',
 'DEPT OF GENERAL SERVI',
 'DEPT RE-CITY OF NY']

In [6]:
property_data = data[~data['OWNER'].isin(remove_list)].reset_index(drop=True) 

In [7]:
property_data.shape

(1046826, 32)

In [8]:
data.shape

(1070994, 32)

In [9]:
len(data)-len(property_data)

24168

In [10]:
missing_zips=np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

21772

In [11]:
for i in range(len(missing_zips)):
    if (property_data.loc[missing_zips[i]+1,'ZIP']==property_data.loc[missing_zips[i]-1,'ZIP']):
        property_data.loc[missing_zips[i],'ZIP']= property_data.loc[missing_zips[i]-1,'ZIP']

In [12]:
missing_zips=np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

10245

In [13]:
for i in range(len(missing_zips)):
    property_data.loc[missing_zips[i],'ZIP']=property_data.loc[missing_zips[i]-1,'ZIP']

In [14]:
missing_zips=np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

0

In [15]:
temp=property_data[property_data['STORIES'].isnull()]

In [16]:
len(temp)

43968

In [17]:
temp['TAXCLASS'].value_counts()

1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [18]:
mean_stories=property_data.groupby('TAXCLASS')['STORIES'].mean()
print(mean_stories)

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64


In [19]:
len(property_data[property_data["STORIES"]==0])

0

In [20]:
property_data['STORIES']=property_data['STORIES'].fillna(value=0)
for index in mean_stories.index:
    property_data.loc[(property_data["STORIES"]==0)&(property_data["TAXCLASS"]==index),'STORIES']=mean_stories[index]

In [21]:
property_data['FULLVAL'].replace('NAN',0)
temp=property_data[property_data['FULLVAL']!=0]
mean_fullval=temp.groupby('TAXCLASS')['FULLVAL'].mean()
print(mean_fullval)

TAXCLASS
1     5.700059e+05
1A    3.352942e+05
1B    3.738399e+05
1C    7.615359e+05
1D    2.233614e+07
2     8.005832e+05
2A    8.639066e+05
2B    1.254525e+06
2C    7.723493e+05
3     8.510005e+04
4     2.772747e+06
Name: FULLVAL, dtype: float64


In [22]:
property_data['AVLAND'].replace('NAN',0)
temp_avland=property_data[property_data['AVLAND']!=0]
mean_avland=temp_avland.groupby('TAXCLASS')['AVLAND'].mean()

In [23]:
property_data['AVTOT'].replace('NAN',0)
temp_avtot=property_data[property_data['AVTOT']!=0]
mean_avtot=temp_avtot.groupby('TAXCLASS')['AVTOT'].mean()

In [24]:
%%time
for index in mean_fullval.index:
    property_data.loc[(property_data['FULLVAL']==0)&(property_data['TAXCLASS']==index),'FULLVAL']=mean_fullval[index]
    property_data.loc[(property_data['AVLAND']==0)&(property_data['TAXCLASS']==index),'AVLAND']=mean_avland[index]
    property_data.loc[(property_data['AVTOT']==0)&(property_data['TAXCLASS']==index),'AVTOT']=mean_avtot[index]
    

CPU times: user 1.88 s, sys: 15.2 ms, total: 1.89 s
Wall time: 1.85 s


In [25]:
property_data.loc[property_data['LTFRONT']==0,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==0,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==0,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==0,'BLDDEPTH']=np.nan
property_data.loc[property_data['LTFRONT']==1,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==1,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==1,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==1,'BLDDEPTH']=np.nan

In [26]:
mean_LTFRONT=property_data.groupby(property_data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH=property_data.groupby(property_data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT=property_data.groupby(property_data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH=property_data.groupby(property_data['TAXCLASS'])['BLDDEPTH'].mean()

In [27]:
for index in mean_LTFRONT.index:
    property_data.loc[(property_data['LTFRONT'].isnull())&(property_data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]    
    property_data.loc[(property_data['LTDEPTH'].isnull())&(property_data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    property_data.loc[(property_data['BLDFRONT'].isnull())&(property_data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    property_data.loc[(property_data['BLDDEPTH'].isnull())&(property_data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]


In [28]:
data=property_data.copy()
data.shape

(1046826, 32)

In [29]:
data['ZIP']=data['ZIP'].astype(str)
data['zip3']=data['ZIP'].str[:3]

In [30]:
data['ltsize']=data['LTFRONT']*data['LTDEPTH']
data['bldsize']=data['BLDFRONT']*data['BLDDEPTH']
data['bldvol']=data['bldsize']*data['STORIES']

In [31]:
data['r1']=data['FULLVAL']/data['ltsize']
data['r2']=data['FULLVAL']/data['bldsize']
data['r3']=data['FULLVAL']/data['bldvol']
data['r4']=data['AVLAND']/data['ltsize']
data['r5']=data['AVLAND']/data['bldsize']
data['r6']=data['AVLAND']/data['bldvol']
data['r7']=data['AVTOT']/data['ltsize']
data['r8']=data['AVTOT']/data['bldsize']
data['r9']=data['AVTOT']/data['bldvol']

In [32]:
ninevars=['r1','r2','r3','r4','r5','r6','r7','r8','r9']


In [33]:
zip5_mean=data.groupby('ZIP')[ninevars].mean()
zip3_mean=data.groupby('zip3')[ninevars].mean()
taxclass_mean=data.groupby('TAXCLASS')[ninevars].mean()
borough_mean=data.groupby('BORO')[ninevars].mean()
data=data.join(zip5_mean,on='ZIP',rsuffix='_zip5')
data=data.join(zip3_mean,on='zip3',rsuffix='_zip3')
data=data.join(taxclass_mean,on='TAXCLASS',rsuffix='_taxclass')
data=data.join(borough_mean,on='BORO',rsuffix='_boro')
rsuffix=['_zip5','_zip3','_taxclass','_boro']

In [34]:
zip5_mean

,r1,r2,r3,r4,r5,r6,r7,r8,r9
ZIP,,,,,,,,,
10001.0,549.795206,716.883268,136.984201,82.637842,109.740911,26.541252,209.814732,268.984579,47.935120
10002.0,352.125998,448.615898,107.921680,40.427291,50.976960,12.851459,110.406623,139.095791,30.521225
10003.0,588.887449,797.964200,178.196506,44.231332,59.227425,13.429450,131.456157,171.798366,31.681455
10004.0,426.099319,501.101842,43.695814,56.095364,69.534410,5.918673,178.639520,211.827721,16.577235
10005.0,130.391459,151.288260,9.069592,20.180111,22.832907,1.587229,57.196453,66.453250,3.754656
...,...,...,...,...,...,...,...,...,...
11694.0,141.422130,605.907054,315.606183,5.881496,20.047830,10.725211,10.995764,37.184449,19.661139
11695.0,108.191259,12929.163544,6682.080001,31.701018,2923.159956,1013.756415,33.058478,3285.953190,1213.643399
11696.0,30.178792,46.934346,8.506826,13.580456,21.120456,3.828072,13.580456,21.120456,3.828072


In [35]:
for var in ninevars:
    for r in rsuffix:
        data[str(var)+r]=data[var]/data[str(var)+r]
#         data[str(var)+r] = data[var]/data[str(var)+r]
        
    

In [36]:
data.columns

Index(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro'],
      dtype='object')

In [37]:
data=data[[ 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro']]

In [38]:
data.head()

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r1_zip5,...,r9_taxclass,r1_boro,r2_boro,r3_boro,r4_boro,r5_boro,r6_boro,r7_boro,r8_boro,r9_boro
0,40.917782,3994.412456,723.985188,8.079350,788.709805,142.953243,18.413002,1797.485605,325.793334,0.096029,...,4.715588,0.113902,7.712077,6.469193,0.241395,16.606462,13.685555,0.198288,14.042014,14.957255
1,57678.882190,36173.697854,6556.463990,4258.951518,2671.030012,484.122805,25955.496986,16278.164034,2950.408795,135.364878,...,42.704717,160.560217,69.841147,58.585497,127.249178,56.239136,46.347248,279.513149,127.165526,135.454020
2,385.182796,736.308714,14.726174,62.110738,118.729804,2.374596,173.332268,331.338940,6.626779,0.903974,...,0.095917,1.072230,1.421603,0.131586,1.855748,2.499883,0.227331,1.866605,2.588430,0.304237
3,556.662267,1064.105879,21.282118,89.761892,171.587267,3.431745,250.498069,478.847739,9.576955,1.306414,...,0.138618,1.549576,2.054487,0.190167,2.681911,3.612808,0.328536,2.697598,3.740773,0.439680
4,1035.027624,1978.540749,39.570815,94.470682,180.588507,3.611770,465.762431,890.343337,17.806867,2.429076,...,0.257740,2.881198,3.820001,0.353586,2.822600,3.802332,0.345771,5.015767,6.955390,0.817518


In [39]:
len(data.columns)

45

In [40]:
round(data.describe().transpose(),2)

,count,mean,std,min,25%,50%,75%,max
r1,1046826.0,213.57,532.56,0.0,76.26,152.86,243.18,138637.34
r2,1046826.0,536.42,1027.44,0.0,211.82,500.00,683.82,310551.57
r3,1046826.0,244.08,498.82,0.0,82.50,239.06,336.23,227500.00
r4,1046826.0,10.46,69.98,0.0,2.35,4.60,7.23,22249.80
r5,1046826.0,22.82,378.51,0.0,6.20,14.98,20.29,334636.37
r6,1046826.0,10.04,178.33,0.0,2.02,7.15,10.04,111545.46
r7,1046826.0,26.83,203.87,0.0,5.28,8.58,13.53,64711.85
r8,1046826.0,50.46,932.20,0.0,17.47,26.98,36.00,871362.22
r9,1046826.0,19.07,327.02,0.0,7.09,12.68,17.58,290454.07
r1_zip5,1046826.0,1.00,2.16,0.0,0.46,0.92,1.23,747.76


In [41]:
stats=round(data.describe().transpose(),2)

In [42]:
stats.to_excel('stats_on_vars.xlsx')